In [65]:
import numpy as np
import pandas as pd
import re

[Requête SQL data.stackexchange.com ici](https://data.stackexchange.com/stackoverflow/query/1824062/top-50k-relevant-questions)

⇒ compromis fait entre qualité maximale et contraintes (≥ 5 tags)
- pour qualité, score ≥ 10 et > 3 réponses

🚧 >=1 réponse : voir cohérence & qualité

- dates jusqu’à 2009, cohérent
- reste à spectre large pour tout type de question

seules features indispensables & RGPD dans la requête

In [66]:
df_raw = pd.read_csv("data/2024-03-11 QueryResults.csv")

# Nettoyage des données

In [67]:
# remove useless features
df_raw.drop(columns=["Score", "AnswerCount", "CreationDate"], inplace=True)
# change Tags string to list
df_raw["Tags"] = df_raw["Tags"].apply(lambda x: x[1:-2].split("><"))

df_raw

,Title,Body,Tags
0,std::shared_mutex::unlock_shared() blocks even...,<p>My team has encountered a deadlock that I s...,"[c++, windows, multithreading, stl, shared-loc]"
1,"What is the correct output of sizeof(""string"")?","<p>On a microcontroller, in order to avoid loa...","[c, language-lawyer, sizeof, string-literals, ..."
2,Problem loading external scripts like jQuery,<p>I'm facing a problem since this morning wit...,"[javascript, html, jquery, google-apps-script,..."
3,Does a const char* literal string persistently...,<p>I have functions like the following:</p>\n<...,"[c++, constants, constexpr, lifetime, null-ter..."
4,WillPopScope is deprecated in Flutter,<p>'WillPopScope' is deprecated and shouldn't ...,"[flutter, dart, mobile, cross-platform, deprec..."
...,...,...,...
49995,The activator for bundle is invalid,<p>I'm trying to create a simple plugin in ecl...,"[java, eclipse, eclipse-plugin, osgi, osgi-bundl]"
49996,Pre-allocate space for C++ STL queue,<p>I'm writing a radix sort algorithm using qu...,"[c++, performance, memory, stl, queu]"
49997,Adjusting the positions of the labels in a UIT...,<p>I'm using a UITableViewCell with UITableVie...,"[iphone, objective-c, cocoa-touch, iphone-sdk-..."
49998,Padding on StackPanel?,<p>I am trying to set padding of a StackPanel ...,"[wpf, xaml, layout, padding, stackpane]"


🚧 [HTML Parser](https://stackoverflow.com/questions/59660495/python-html-extract-text-into-list) : (cf. [question SoF](https://stackoverflow.com/questions/59660495/python-html-extract-text-into-list))  
[RegEx JL](https://www.notion.so/julmat/RegEx-expressions-r-guli-res-70fbbcb177ee476ba9a5ae011d14fe6f) & [RegEx 101](https://regex101.com/?flavor=python&flags=gm)

In [68]:
ex_body = df_raw["Body"][0]

- suppr chiffres et caractères **seuls** " 123 " ou "."
- suppr certains mots d'1 caractère (pas "c", "b", "r"...) => voir liste dans les tags déjà présents ? récupérer [liste des langages de programmation](https://en.wikipedia.org/wiki/List_of_programming_languages) (cf. [API](https://stackoverflow.com/questions/7185288/how-can-i-get-wikipedia-content-using-wikipedias-api)) ?
- suppr mots 1 alpha + 1 chiffre
- suppr points à la fin d'un mot
- ne pas supprimer "+", "#" et autres caractères utiles (noms de langages)

In [81]:
import urllib.request, json 
with urllib.request.urlopen("https://en.wikipedia.org/w/api.php?action=parse&page=Timeline_of_programming_languages&prop=text&format=json&disabletoc=1&formatversion=2") as url:
    prog_langs = json.load(url)["parse"]["text"]

print(prog_langs)

<div class="mw-content-ltr mw-parser-output" lang="en" dir="ltr"><p class="mw-empty-elt">
</p>
<style data-mw-deduplicate="TemplateStyles:r1097763485">.mw-parser-output .ambox{border:1px solid #a2a9b1;border-left:10px solid #36c;background-color:#fbfbfb;box-sizing:border-box}.mw-parser-output .ambox+link+.ambox,.mw-parser-output .ambox+link+style+.ambox,.mw-parser-output .ambox+link+link+.ambox,.mw-parser-output .ambox+.mw-empty-elt+link+.ambox,.mw-parser-output .ambox+.mw-empty-elt+link+style+.ambox,.mw-parser-output .ambox+.mw-empty-elt+link+link+.ambox{margin-top:-1px}html body.mediawiki .mw-parser-output .ambox.mbox-small-left{margin:4px 1em 4px 0;overflow:hidden;width:238px;border-collapse:collapse;font-size:88%;line-height:1.25em}.mw-parser-output .ambox-speedy{border-left:10px solid #b32424;background-color:#fee7e6}.mw-parser-output .ambox-delete{border-left:10px solid #b32424}.mw-parser-output .ambox-content{border-left:10px solid #f28500}.mw-parser-output .ambox-style{border-l

In [69]:
def clean_string(string):
    """Return a cleaned version of an input string"""
    # remove code tags
    string = re.sub(r"<code>.*?<\/code>", "", string, flags=re.S)
    # remove img tags
    string = re.sub(r"<img.*?>", "", string)
    # remove all html tags
    string = re.sub(r"<.*?>", "", string)
    # remove newlines
    string = re.sub(r"\n", " ", string)
    # lowercase
    string = string.lower()
    # keep only letters, digits, spaces and some useful characters
    string = re.sub(r"[^\w _.-]", "", string)
    # remove multiple spaces
    string = re.sub(r" +", " ", string)

    return string

df_raw["Body"] = df_raw["Body"].apply(clean_string)
df_raw["Title"] = df_raw["Title"].apply(clean_string)

In [70]:
df_raw

,Title,Body,Tags
0,stdshared_mutexunlock_shared blocks even thoug...,my team has encountered a deadlock that i susp...,"[c++, windows, multithreading, stl, shared-loc]"
1,what is the correct output of sizeofstring,on a microcontroller in order to avoid loading...,"[c, language-lawyer, sizeof, string-literals, ..."
2,problem loading external scripts like jquery,im facing a problem since this morning with we...,"[javascript, html, jquery, google-apps-script,..."
3,does a const char literal string persistently ...,i have functions like the following and im pla...,"[c++, constants, constexpr, lifetime, null-ter..."
4,willpopscope is deprecated in flutter,willpopscope is deprecated and shouldnt be use...,"[flutter, dart, mobile, cross-platform, deprec..."
...,...,...,...
49995,the activator for bundle is invalid,im trying to create a simple plugin in eclipse...,"[java, eclipse, eclipse-plugin, osgi, osgi-bundl]"
49996,pre-allocate space for c stl queue,im writing a radix sort algorithm using queues...,"[c++, performance, memory, stl, queu]"
49997,adjusting the positions of the labels in a uit...,im using a uitableviewcell with uitableviewcel...,"[iphone, objective-c, cocoa-touch, iphone-sdk-..."
49998,padding on stackpanel,i am trying to set padding of a stackpanel but...,"[wpf, xaml, layout, padding, stackpane]"


In [71]:
print(ex_body)
print(df_raw["Body"][0])

<p>My team has encountered a deadlock that I suspect is a bug in the Windows implementation of SRW locks. The code below is a distilled version of real code. Here's the summary:</p>
<ol>
<li>Main thread acquires exclusive lock</li>
<li>Main thread creates N children threads</li>
<li>Each child thread
<ol>
<li>Acquires a shared lock</li>
<li>Spins until all children have acquired a shared lock</li>
<li>Releases the shared lock</li>
</ol>
</li>
<li>Main thread releases exclusive lock</li>
</ol>
<p>Yes this could be done with std::latch in C++20. That's not the point.</p>
<p>This code works <em>most</em> of the time. However roughly 1 in 5000 loops it deadlocks. When it deadlocks exactly 1 child successfully acquires a shared lock and N-1 children are stuck in <code>lock_shared()</code>. On Windows this function calls into <code>RtlAcquireSRWLockShared</code> and blocks in <code>NtWaitForAlertByThreadId</code>.</p>
<p>The behavior is observed when used <code>std::shared_mutex</code> direc

In [72]:
display(*df_raw["Title"][:20].values)

'stdshared_mutexunlock_shared blocks even though there are no active exclusive locks on windows'

'what is the correct output of sizeofstring'

'problem loading external scripts like jquery'

'does a const char literal string persistently exist as long as the process alive'

'willpopscope is deprecated in flutter'

...

'compilation error after upgrading to jdk 21 - nosuchfielderror jcimport does not have member field jctree qualid'

'ld warning ignoring duplicate libraries -lgcc after the recent update of xcodebuild tools'

'error encountered only below ios 17 devices thread 1 exc_bad_access code1 address0x0'

'handling cycle inside runner building could produce unreliable results after update xcode to xcode 15'

'what is the purpose of _t aliases and _v variable templates for type traits'